## BOOSTING MACHINES

In [1]:
data_file=r'diabetic_data.csv'

import pandas as pd
import numpy as np

dd=pd.read_csv(data_file)
dd['readmitted']=np.where(np.logical_or(dd['readmitted']=="NO", 
                                        dd['readmitted']==">30"),
                          "NO","YES")
race_dummies=pd.get_dummies(dd['race'])
race_dummies.columns=["race_"+x for x in race_dummies.columns]

dd=pd.concat([dd,race_dummies],1)
drop_vars=["race_?","race"]

dd['gender_F']=np.where(dd['gender']=="Female",1,0)
drop_vars=drop_vars+['gender']

dd['age1'],dd['age2']=zip(*dd['age'].apply(lambda x: x.split('-', 1)))


dd['age1']=[x.replace('[',"") for x in dd['age1']]
dd['age2']=[x.replace(')',"") for x in dd['age2']]

dd['age']=0.5*(pd.to_numeric(dd['age1'])+pd.to_numeric(dd['age2']))

drop_vars=drop_vars+['age1','age2']

drop_vars+=['weight']

dd['admission_type_id'].value_counts()
ad_type_dummies=pd.get_dummies(dd['admission_type_id'])
ad_type_dummies.columns=["ad_type_"+str(x) for x in 
                         ad_type_dummies.columns]
dd=pd.concat([dd,ad_type_dummies],1)
drop_vars+=['ad_type_4']+['admission_type_id']

dd['discharge_disposition_id'].value_counts()
dis_dispo_dummies=pd.get_dummies(dd['discharge_disposition_id'])
dis_dispo_dummies.columns=['dis_dispo_'+str(x) for x 
                           in dis_dispo_dummies.columns]
dd=pd.concat([dd,dis_dispo_dummies],1)
drop_vars+=['dis_dispo_'+str(x) for x in 
            [15,24,9,17,16,19,10,27,12,20]]+['discharge_disposition_id']


dd['admission_source_id'].value_counts()
ad_source_dummies=pd.get_dummies(dd['admission_source_id'])
ad_source_dummies.columns=['ad_source_'+str(x) for 
                           x in ad_source_dummies.columns]
dd=pd.concat([dd,ad_source_dummies],1)
drop_vars+=['ad_source_'+str(x) for x in 
            [8,22,10,11,14,25,13]]+['admission_source_id']

dd['payer_code'].value_counts()
payer_code_dummies=pd.get_dummies(dd['payer_code'])
payer_code_dummies.columns=['payer_code_'+str(x) for 
                            x in payer_code_dummies.columns]
dd=pd.concat([dd,payer_code_dummies],1)
drop_vars+=['payer_code_'+str(x) for x in 
            ['OT','MP','SI','FR']]+['payer_code']

dd['medical_specialty'].value_counts()
for ms in ['?','InternalMedicine','Emergency/Trauma',
           'Family/GeneralPractice','Cardiology','Surgery-General',
          'Nephrology','Orthopedics','Orthopedics-Reconstructive',
           'Radiologist']:
    v_name=ms.replace("?","NoRecord").replace("/","_").replace("-","_")
    dd['ms_'+v_name]=np.where(dd['medical_specialty']==ms,1,0)

    drop_vars+=['medical_specialty']+['diag_1','diag_2','diag_3',
                                  'examide','citoglipton']


dd=dd.drop(drop_vars,1)
drop_vars=[]

for col in dd.columns:
    if(dd[col].dtype=="object"):
        temp=pd.get_dummies(dd[col],drop_first=True)
        temp.columns=[col+'_'+str(x) for x in temp.columns]
        drop_vars+=[col]
        dd=pd.concat([dd,temp],1)
#         print(col)
dd=dd.drop(drop_vars,1)


dd.columns=[x.replace("-","_").replace("?","_") for x in dd.columns]

from sklearn.cross_validation import train_test_split

dd_train, dd_test = train_test_split(dd, test_size = 0.2,random_state=2)
y_train=dd_train['readmitted_YES']
x_train=dd_train.drop(['readmitted_YES','encounter_id','patient_nbr'],1)
y_test=dd_test['readmitted_YES']
x_test=dd_test.drop(['readmitted_YES','encounter_id','patient_nbr'],1)

C:\Users\patilsh\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## ADABOOST

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1,
                    class_weight='balanced',random_state=2),
                         algorithm="SAMME",
                         n_estimators=1000,
                        learning_rate=0.5,
                        random_state=2)


You should realise here that there are two sets of hyperparameter which you can tune here. One is for the weak learner which in this case is a simple tree stump, another is for the booster. Although this being a weak learner too much tuning might not make much difference as such.

In [4]:
bdt.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=2,
            splitter='best'),
          learning_rate=0.5, n_estimators=1000, random_state=2)

In [6]:
from __future__ import division

In [7]:
df=pd.DataFrame(list(zip(y_test,bdt.predict(x_test))),
                columns=['real','predicted'])
TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
P=TP+FN
N=TN+FP

#Accuracy
Acc=(TP+TN)/(P+N)

#Sensitivity
Sn=TP/P

#Specificity
Sp=TN/N

print('Accuracy is :',Acc )
print('Sensitvity is :',Sn )
print('Specificity is :',Sp )

('Accuracy is :', 0.6654711604598604)
('Sensitvity is :', 0.5071003206596427)
('Specificity is :', 0.6844972758791481)


## XGBOOST

In [8]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import RandomizedSearchCV


clf=XGBClassifier(objective="binary:logistic",silent=False,
                  scale_pos_weight=9)

C:\Users\patilsh\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |      Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of boosted trees to fit.
 |  silent : boolean
 |      Whether to print messages while running boosting.
 |  objective : string or callable
 |      Specify the learning task and the corresponding learning objective or
 |      a custom objective function to be used (see note below).
 |  nthread : int
 |      Number of parallel threads used to run xgboost.
 |  gamma : float
 |      Minimum loss reduction required to make a further partition on a leaf node of the tree.
 |  min_child_weight : int
 |      Minimum sum of instance weight(hessian) needed in a child.
 |  max_delta_step : int
 |      Ma

In [15]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
# specify parameters and distributions to sample from
param_dist = {
              "max_depth": [2,3,4,5,6],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000]
              }
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(x_train, y_train)
report(random_search.grid_scores_)

Model with rank: 1
Mean validation score: 0.795 (std: 0.002)
Parameters: {'gamma': 0.1, 'colsample_bytree': 0.8, 'subsample': 0.6, 'n_estimators': 1000, 'reg_alpha': 0.1, 'max_depth': 6, 'learning_rate': 0.3, 'min_child_weight': 6}

Model with rank: 2
Mean validation score: 0.694 (std: 0.003)
Parameters: {'gamma': 0.1, 'colsample_bytree': 0.9, 'subsample': 0.6, 'n_estimators': 500, 'reg_alpha': 0.1, 'max_depth': 6, 'learning_rate': 0.1, 'min_child_weight': 5}

Model with rank: 3
Mean validation score: 0.689 (std: 0.004)
Parameters: {'gamma': 0.3, 'colsample_bytree': 0.8, 'subsample': 0.9, 'n_estimators': 500, 'reg_alpha': 0.1, 'max_depth': 4, 'learning_rate': 0.5, 'min_child_weight': 4}



This ran for over two hours on my machine , i think. Why i am not sure because i left it running and went to sleep. Anyways, point is when you are tuning for hyperparameters , be ready for not getting quick results. Its part of the process.

Lets now select the top performer here and build our model with that.

In [16]:
clf=XGBClassifier(objective="binary:logistic",silent=False,gamma=0.1,
                  colsample_bytree=0.8,
                  subsample=0.6,n_estimators=1000,
                  reg_alpha=0.1,max_depth=6,learning_rate=0.3,
                 min_child_weight=6,scale_pos_weight=9)

In [17]:
clf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.1, learning_rate=0.3, max_delta_step=0, max_depth=6,
       min_child_weight=6, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=1,
       scale_pos_weight=9, seed=0, silent=False, subsample=0.6)

In [18]:
df=pd.DataFrame(list(zip(y_test,clf.predict(x_test))),
                columns=['real','predicted'])
TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
P=TP+FN
N=TN+FP

#Accuracy
Acc=(TP+TN)/(P+N)

#Sensitivity
Sn=TP/P

#Specificity
Sp=TN/N

print('Accuracy is :',Acc )
print('Sensitvity is :',Sn )
print('Specificity is :',Sp )

Accuracy is : 0.7689397661393338
Sensitvity is : 0.30737517178195145
Specificity is : 0.8243905123548512


This gives you a good boost in sensitivity but at the same time , doesnt hamper specificity much. We are calling this a good boost , because other algorithms were performing very poorly on the same data.
